In [132]:
from __future__ import division
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

In [159]:
ratings_train = pd.read_csv('./homework4/data/ratings_train.csv', header=None)
ratings_test = pd.read_csv('./homework4/data/ratings_test.csv', header=None)
ratings_fake = pd.read_csv('./homework4/data/ratings_fake.csv', header=None)

In [160]:
ratings_train.columns = ['user', 'movie', 'rating']
ratings_test.columns = ['user', 'movie', 'rating']
ratings_fake.columns = ['user', 'movie', 'rating']

In [161]:
A = ratings_fake.pivot(index='user', columns='movie').as_matrix()

In [309]:
mu = A.mean()
m = A.shape[0]
n = A.shape[1]
k = 5
T = 20
B = np.zeros(m)
C = np.zeros(n)
U = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (m))
V = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (n))

In [310]:
U = optimize_U(U, V, B, C, mu, A)

In [268]:
# V = optimize_V(U, V, B, C, mu, A)

In [315]:
def optimize_U(U, V, B, C, mu, A):
    rights = []
    for i in range(U.shape[0]):
        js = []
        for j in range(V.shape[0]):
            r = -(B[i] + C[j] + mu - A[i, j]) * V[j]
            js.append(r)
        rights.append(np.array(js).sum(axis=0))
    rights = np.array(rights)

    vec_matrix = []
    for i in range(U.shape[0]):
        lefts = 0
        for j in range(V.shape[0]):
            arr = np.outer(V[j], V[j].T)
            lefts += arr
        lefts = np.linalg.inv(np.array(lefts))
        vec_matrix.append(lefts)
    vec_matrix = np.array(vec_matrix)

    vecs = []
    for i in range(200):
        new_vec_i = np.dot(vec_matrix[i], rights[i])
        vecs.append(new_vec_i)

    return np.array(vecs)

In [316]:
def optimize_V(U, V, B, C, mu, A):
    rights = []
    for i in range(U.shape[0]):
        js = []
        for j in range(V.shape[0]):
            r = -(B[i] + C[j] + mu - A[i, j]) * U[i]
            js.append(r)
        rights.append(np.array(js).sum(axis=0))
    rights = np.array(rights)

    vec_matrix = []
    for i in range(U.shape[0]):
        lefts = 0
        for j in range(V.shape[0]):
            arr = np.outer(U[j], U[j].T)
            lefts += arr
        lefts = np.linalg.inv(np.array(lefts))
        vec_matrix.append(lefts)
    vec_matrix = np.array(vec_matrix)

    vecs = []
    for i in range(U.shape[0]):
        new_vec_i = np.dot(vec_matrix[i], rights[i])
        vecs.append(new_vec_i)

    return np.array(vecs)

In [317]:
def optimize_B(U, V, B, C, mu, A):
    sums = []
    for i in range(U.shape[0]):
        js = []
        for j in range(V.shape[0]):
            s = -(np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i, j]) / V.shape[0]
            js.append(s)
        sums.append(np.array(js).sum(axis=0))
    return np.array(sums)

In [318]:
def optimize_C(U, V, B, C, mu, A):
    sums = []
    for i in range(U.shape[0]):
        js = []
        for j in range(V.shape[0]):
            s = -(np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i, j]) / U.shape[0]
            js.append(s)
        sums.append(np.array(js).sum(axis=0))
    return np.array(sums)

In [319]:
def update(U, V, B, C, mu, A):
    sums = []
    for i in range(U.shape[0]):
        for j in range(V.shape[0]):
            s = -(np.dot(U[i], V[j]) + B[i] + C[j] + mu - A[i,j])
            sums.append(s)
    sums = np.array(sums)
    return sums.sum()

In [320]:
def optimize_params(U, V, B, C):
    B = optimize_B(U, V, B, C, mu, A)
    U = optimize_U(U, V, B, C, mu, A)
    C = optimize_C(U, V, B, C, mu, A) 
    V = optimize_V(U, V, B, C, mu, A)
    return (B, U, C, V)

In [321]:
# B = optimize_B(U, V, B, C, mu, A)
# U = optimize_U(U, V, B, C, mu, A)
# C = optimize_C(U, V, B, C, mu, A) 
# V = optimize_V(U, V, B, C, mu, A)

In [322]:
mu = A.mean()
m = A.shape[0]
n = A.shape[1]
k = 5
T = 20
B = np.zeros(m)
C = np.zeros(n)
U = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (m))
V = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (n))

In [323]:
for t in range(T):
    B, U, C, V = optimize_params(U, V, B, C)
    print update(U, V, B, C, mu, A)

6.46402343898
0.0156186124211
0.702202398466
-0.032858499304
0.128136942082
-2.15038778109
49.8472105143
68.7773933782
263.371463894


KeyboardInterrupt: 

In [448]:
lefts = []
for j in range(V.shape[0]):
    arr = np.dot(V[j], V[j].T)
    lefts.append(arr)
lefts = np.array(lefts)
lefts = np.linalg.inv([[lefts.sum()]])

In [170]:
right = []
for i in range(U.shape[0]):
    js = []
    for j in range(V.shape[0]):
        r = (B[i] + C[j] + mu - A[i, j]) * V[j]
        js.append(r)
    right.append(np.array(js).sum(axis=0))
right = np.array(right)

In [207]:
# def non_u_component(U, B, C, mu, A):
#     new_vec = []
#     for i in range(U.shape[0]):
#         s = (B[i] + C + mu - A[i, :])
#         new_vec.append(s)
#         nuc = np.array(new_vec)
#     return nuc * V

In [166]:
# def non_v_component(U, B, C, mu, A):
#     new_vec = []
#     for i in range(V.shape[0]):
#         s = (B[i] + C + mu - A[i, :]) * V
#         new_vec.append(s)
#         nc = np.array(new_vec)
#     return nc